<a href="https://colab.research.google.com/github/marcpadro/eines_colab/blob/main/Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown # Inicialitzar

# !pip install torch torchvision
# !pip install git+https://github.com/huggingface/transformers@335c0ca35c159f88d73198bdac928e61a4d480c7
# !pip install transformers
!pip install peft
# !pip install accelerate
# !pip install -i https://pypi.org/simple/ bitsandbytes

#Import Hugging Face's Transformers
from transformers import pipeline
# This is to log our outputs in a nicer format
from pprint import pprint
import os
from peft import PeftModel, PeftConfig
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import gc
import torch
import time

In [ ]:
#@markdown # Carregar Model

entrenament = 'punk' #@param ["punk", "manifestos", "antic-testament", "punk-gran", "manifestos-gran"]

# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

if not os.path.exists(entrenament):
  !wget 'https://tallerestampa.com/files/models/{entrenament}.zip'
  !unzip {entrenament}.zip


try:
  del config
  del model
except:
  pass
try:
  del tokenizer
  del lm
  del generator
except:
  pass
time.sleep(0.1)
gc.collect()

torch.cuda.empty_cache()

# gc.collect()
# torch.cuda.empty_cache()

# time.sleep(1)

if entrenament in ('punk-gran', 'manifestos-gran'):
  config = PeftConfig.from_pretrained(entrenament)
  model = AutoModelForCausalLM.from_pretrained(
      config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map="auto"
  )
  tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, padding_side="left")
  # Load the Lora model
  lm = PeftModel.from_pretrained(model, entrenament)
  generator = pipeline('text-generation', model=lm, tokenizer=tokenizer)
else:
  lm = AutoModelForCausalLM.from_pretrained(entrenament)
  tokenizer = AutoTokenizer.from_pretrained(entrenament, padding_side="left")
  generator = pipeline('text-generation', model=lm, tokenizer=tokenizer, device=0)

In [ ]:
#@title Your first completion

prompt = "My name is Zack and I like to"#@param {type: "string"}
min_length = 50#@param {type: "number"}
max_length = 200#@param {type: "number"}
temperature = 0.8#@param {type: "slider", min: 0, max: 4, step: 0.1}
temperature = max(0.0001, temperature)
output = generator(prompt, do_sample=True, min_length=min_length, max_length=max_length, temperature=temperature)
print()
print()
print( output[0]['generated_text'] )